# 🔵 Task 11: YOLOv8 Training for Waste Classification

## 🎯 Objective
Train a YOLOv8 model on the same waste classification dataset and compare with YOLOv11.

---

## 📚 YOLOv8 Overview

| Feature | YOLOv8 |
|---------|--------|
| Release | 2023 (Ultralytics) |
| Backbone | Modified CSPDarknet |
| Neck | C2f + PANet |
| Head | Decoupled, Anchor-free |
| Key Innovation | C2f block, unified framework |

### Differences from YOLOv11:
- Uses **C2f** blocks instead of **C2PSA**
- Slightly fewer params at same scale
- Well-tested, large community support

In [ ]:
import os
from pathlib import Path
import yaml
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from ultralytics import YOLO

PROJECT_ROOT = Path(r"D:\het\SELF\RP\YOLO-V11-PRO")
DATA_DIR = PROJECT_ROOT / "data" / "processed"
MODELS_DIR = PROJECT_ROOT / "models"
MODELS_DIR.mkdir(exist_ok=True)

print("✅ Setup complete!")

---

# Part 1: Load YOLOv8 Model

In [ ]:
# Load pre-trained YOLOv8 nano
model = YOLO('yolov8n.pt')

print("✅ YOLOv8 Nano Model Loaded!")
print(f"   Pre-trained on: COCO (80 classes)")
print(f"   Fine-tuning for: Waste (2 classes)")
model.info()

---

# Part 2: Training Configuration

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
run_name = f"waste_yolov8n_{timestamp}"

training_args = {
    'data': str(DATA_DIR / 'dataset.yaml'),
    'epochs': 100,
    'patience': 20,
    'batch': 16,
    'imgsz': 640,
    'optimizer': 'AdamW',
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'augment': True,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 10,
    'translate': 0.1,
    'scale': 0.5,
    'flipud': 0.5,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.1,
    'project': str(MODELS_DIR),
    'name': run_name,
    'exist_ok': True,
    'device': 0,
    'workers': 4,
    'amp': True,
    'verbose': True,
    'plots': True,
    'save': True,
    'save_period': 10,
}

print(f"📋 YOLOv8 Config: {run_name}")
for k, v in training_args.items():
    print(f"   {k:<20} {str(v):<30}")

---

# Part 3: Train

In [ ]:
print("🚀 STARTING YOLOv8 TRAINING")
results = model.train(**training_args)
print("✅ TRAINING COMPLETE!")

---

# Part 4: Evaluate

In [ ]:
best_model_path = MODELS_DIR / run_name / 'weights' / 'best.pt'

if best_model_path.exists():
    best_model = YOLO(str(best_model_path))
    val_results = best_model.val(data=str(DATA_DIR / 'dataset.yaml'))
    
    print("📊 YOLOv8 Validation Metrics:")
    print(f"   mAP@50:    {val_results.box.map50:.4f}")
    print(f"   mAP@50-95: {val_results.box.map:.4f}")
    print(f"   Precision: {val_results.box.mp:.4f}")
    print(f"   Recall:    {val_results.box.mr:.4f}")

In [ ]:
# Test inference
from PIL import Image

test_images = list((DATA_DIR / 'images' / 'val').glob('*.jpg'))[:6]

if best_model_path.exists() and test_images:
    best_model = YOLO(str(best_model_path))
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('🔵 YOLOv8 Predictions', fontsize=14, fontweight='bold')
    
    for ax, img_path in zip(axes.flat, test_images):
        results = best_model.predict(source=str(img_path), conf=0.25, save=False, verbose=False)
        ax.imshow(results[0].plot())
        ax.set_title(img_path.name[:25])
        ax.axis('off')
    
    plt.tight_layout()
    plt.savefig(PROJECT_ROOT / 'docs' / 'assets' / 'yolov8_predictions.png', dpi=150)
    plt.show()

In [ ]:
# Export
if best_model_path.exists():
    best_model = YOLO(str(best_model_path))
    onnx_path = best_model.export(format='onnx')
    print(f"✅ Exported: {onnx_path}")

print("\n" + "="*60)
print("✅ TASK 11 COMPLETE: YOLOv8 Training")
print("="*60)